In [2]:
import os
import pandas as pd
from pathlib import Path
import numpy as np

In [ ]:
dataPath = Path("../..") / "validation-meteo-data" / "donneesmeteo_2010-2024" / "donneesmeteo_2010-2024.csv"
meteodf = pd.read_csv(dataPath, sep=";")
meteodf.head()

In [ ]:
print("Parameters:", meteodf.libellecourt.unique())
meteodf.shape

In [ ]:
meteodf.fillna({"valeurorigine": meteodf.valeur}, inplace=True) # NaNs in valeurorigine means that there has been no correction of the original value
meteodf

In [ ]:
meteodf.replace(to_replace={"valeurorigine":-999}, value=np.nan, inplace=True) # -999 can be considered as actual NaNs
meteodf

In [7]:
meteodfToUse = meteodf[meteodf.columns[:5]] # For the classification model we only need the first 5 columns

In [ ]:
meteodfToUse.dropna(inplace=True)


In [9]:
meteodfToUse.reset_index(drop=True, inplace=True)

In [ ]:
meteodfToUse.datemesure = pd.to_datetime(meteodfToUse.datemesure.apply(lambda value: value.split(" ")[0])) # all measurements date at 00:00:00 as hour so we can just drop it

In [ ]:
meteodfToUse["correction"] = (meteodfToUse.valeur != meteodfToUse.valeurorigine).astype(int)
meteodfToUse